# Comparison of Machine Learning methods performance to predict the flowpattern in a nearly horizontal pipeline
### By: Gerardo Vera

M.E. Petroleum Engineer\
email: gev7313@utulsa.edu - gerryac568@gmail.com

## Problem Statement

This project shows a way to classify the flow through a nearly horizontal pipeline, more properly said, between minus and plus ten degrees of inclination. The study will only cover the data recorded using a two inch pipeline. The categories for the classification are five, Dispersed Bubble, Stratified Smooth, Stratified Wavy, Annular and Intermittent flow.

The method for the classification will be machine learning, in this case, a comparison of four methods, the decision tree algorithm, the random forest algorithm, the naive Bayes classifier and the support vector machine techniques.

To demonstrate the degree of accuracy of the methods, a percentage of the data recorded will be used as feed, to train the models, and the rest will be used to test the models results. The data selected will always be selected randomly, so the accuracy may vary amongst several runs of the program, but it shouldn't vary too much.

### Configuration

We start importing the libraries we will use in the study. Numpy is a mathematical library that helps us dealing with numbers and operations. Pandas is a dataframe manager which will help us to import and manipulate the dataset to fit our needs. The sklearn library is focused towards predictive data analysis and machine learning functions, from this one we will import several functions.

From sklearn.tree we import the DecisionTreeClassifier function which is the decision tree itself. We do the same with RandomForestClassifier from sklearn.ensemble. GaussianNB from sklearn.naive_bayes implements the Gaussian Naive Bayes algorithm for classification. The likelihood of the features is assumed to be Gaussian. Confusion_matrix will help us to create automatically confusion matrices and accuracy_score will help us to evaluate the models. And finally we import svm which is the support vector machine.

Matplotlib is the library that will help us to display our results in a more user friendly way (using graphs), seaborn is another library dedicated to plotting data and information and finally, the immediately below line is to display these graphs in the same document.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Decision Tree Algorithm
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#Random Forest Algorithm
from sklearn.ensemble import RandomForestClassifier
#Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
#Support Vector Machine
from sklearn import svm

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

### Data Reading and Cleaning

Now that the configuration is done, we can begin reading the data. First, we read the data directly from the excel file, to a new variable called data1, using the pandas's library command read_excel. We use the keywords "header" and "skiprows" to specify that this is going to be a multiindex dataframe since there are three lines in the excel file with titles and subtitles and to skip the fourth line because it contains the units which are not important for our study.

To start filtering the data rows and columns, we declare a new variable, data2, and we put in it some rows and columns from data1, for this we use the comand iloc, proper of the dataframes. We put all the rows below the row 3170, we got this number by checking the excel file and checking which data points we want to work with. For the columns we use their names to select them, so we specify a level and take the columns in those levels with the names we are specifying.

To further filter the data, we now use the inner diameter column because we are interested in working with the datapoints recorded using a diameter bigger than 0.05 meters which is a little less than two inches. We set only a lower limit because the higher value in this column is precisely two inches.

Finally, the last filter. We position ourselves on the column "Angle", under the subtitle "System Geometry" and under the title "Input Data". Now we extract to a new variable, called data4, the rows which inclination is bigger or equal to -10 and smaller or equal to 10 degrees.

We can now take a look at the five first rows of our new dataset.

In [ ]:
data1 = pd.read_excel("/content/drive/MyDrive/ALCE/DataBasePereyraetal2012.xls", 
                      header = [0, 1, 2], skiprows = [3])
data2 = data1.iloc[data1.index[data1['Input Data', 'Test Number', 'Test Code'].str.contains('Shoham')], 
                   (data1.columns.get_level_values(2)=='Test Code') | 
                   (data1.columns.get_level_values(2)=='P') | 
                   (data1.columns.get_level_values(2)=='Type of liquid') | 
                   (data1.columns.get_level_values(2)=='Type of Gas') | 
                   (data1.columns.get_level_values(2)=='DenL') | (data1.columns.get_level_values(2)=='DenG') | 
                   (data1.columns.get_level_values(2)=='VisL') | (data1.columns.get_level_values(2)=='VisG') | 
                   (data1.columns.get_level_values(2)=='ST') | (data1.columns.get_level_values(2)=='ID') | 
                   (data1.columns.get_level_values(2)=='Ang') | (data1.columns.get_level_values(2)=='Vsl') | 
                   (data1.columns.get_level_values(2)=='Vsg') | 
                   (data1.columns.get_level_values(1)=='Unnamed: 17_level_1') | 
                   (data1.columns.get_level_values(1)=='Unnamed: 18_level_1')]
data3 = data2[(data2[('Input Data','System Geometry','ID')] > 0.05) & 
              (data2[('Input Data','System Geometry','Ang')] >= -10) & 
              (data2[('Input Data','System Geometry','Ang')] <= 10)]
data4 = data3.reset_index(drop = True)
data4.head(10)

We now define a new variable, called categories, and store in it the abbreviations of the flowpatterns.

To split our data into training and testing data, we create a new column called "is_train", in our dataset, and we fill it with boolean data, setting randomly 75 percent of it with the value True and the rest with the value False.

We create two new dataframes, called train and test, and we fill them with the data from our data4 dataframe, depending on the value of the previously created column "is_train".

In [ ]:
categories = ['A', 'DB', 'I', 'SS', 'SW']
np.random.seed(185)
data4['is_train'] = np.random.uniform(0, 1, len(data4)) <= .75
train, test = data4[data4['is_train'] == True], data4[data4['is_train'] == False]
data4

In order to use the naive bayes classifier we need to further split our data, each dataset, train and test, into two new datasets, inputs and outputs. We do this by selecting the different columns of each dataframe. So in the new data2 dataframe, we have the training inputs, in the new data3 dataframe, we have the training outputs, in the new data5 dataframe, we store the testing inputs and in the data6 dataframe, we store the testing outputs.

Finally, we transform the training output dataframe into an array, and then we reshape it into a one dimension vector because that is the input format the function model.fit is expecting. Otherwise we would get a warning message, however, the program would still work as expected.

To check the transformation we display the data stored in the variable data3, but only the first 20 datapoints so it does not take too much space.

In [ ]:
data2 = train.iloc[:, (train.columns.get_level_values(2)=='Vsl') | (train.columns.get_level_values(2)=='Vsg') | (train.columns.get_level_values(2)=='Ang')]
data3 = train.iloc[:, (train.columns.get_level_values(1)=='Unnamed: 17_level_1')]
data5 = test.iloc[:, (test.columns.get_level_values(2)=='Vsl') | (test.columns.get_level_values(2)=='Vsg') | (test.columns.get_level_values(2)=='Ang')]
data6 = test.iloc[:, (test.columns.get_level_values(1)=='Unnamed: 17_level_1')]
data3 = np.asarray(data3)
data3 = data3.reshape(-1, )
data3[0:20]

In [ ]:
y, label = pd.factorize(train['Output', 'Unnamed: 17_level_1', 'Flow Pattern'])
ytest, label = pd.factorize(test['Output', 'Unnamed: 17_level_1', 'Flow Pattern'])
y[0:20]

### Fitting the models

Now that we have input data cleaned, we can train our naive Bayes model. First we create a gaussian model, we use the gaussian naive Bayes method because is the best suited for dealing with numeric inputs. Then we train it with our training input and output datasets.

In [ ]:
#Decision Tree Algorithm
modelDTA = DecisionTreeClassifier(criterion = "entropy", random_state = 100)
modelDTA.fit(data2, data3)
#Random Forest Algorithm
modelRFA = RandomForestClassifier(n_jobs = 2, random_state = 0)
modelRFA.fit(data2, data3)
#Naive Bayes Classifier
modelNBC = GaussianNB()
modelNBC.fit(data2, data3)
#Support Vector Machine
modelSVM = svm.SVC(C = 100)
modelSVM.fit(data2, data3)

We apply the model to the testing inputs and store the results in a variable called predict, we should compare this results with the data stored in the dataframe data6 (output data recorded from experiments) to check the accuracy of our model.

In [ ]:
#Decision Tree Algorithm
predictDTA = modelDTA.predict(data5)
#Random Forest Algorithm
predictRFA = modelRFA.predict(data5)
#Naive Bayes Classifier
predictNBC = modelNBC.predict(data5)
#Support Vector Machine
predictSVM = modelSVM.predict(data5)

In [ ]:
#Decision Tree Algorithm
print(predictDTA[0:25])
print("")
#Random Forest Algorithm
print(predictRFA[0:25])
print("")
#Naive Bayes Classifier
print(predictNBC[0:25])
print("")
#Support Vector Machine
print(predictSVM[0:25])

### Analysing results

To check our results, we create a confusion matrix, comparing the experimental with the predicted data.

To display the result in a more appealing way, we show the confusion matrix as a heatmap, to appreciate the right and wrong predictions by colors.

With the keyword "font_scale" we set the font size of the plot, then with the keyword "figsize" we define the size of the plot. Then we draw the heatmap, using as reference the confusion matrix created before. We set the parameter "square" as true, to show each box of the confusion matrix as a perfect square, "annot" as true to show the flow pattern abbreviation for each row and column, "fmt" as 'd' to show the numbers in decimal format, "cbar" as true to show the color reference and then we match the rows and columns with a variable containing the abbreviations. At last we add the axis labels to show which one is the predicted and which one is the experimental data.

Another way of checking our results is by using the accuracy_score function, which will show us the percentage of correctly predicted flow patterns. This is like dividing the accurate predictions over the wrong prediction in the confusion matrix and multiplying it by a hundred.

In [ ]:
title = ['Decision Tree Algorithm Resulst:', 'Random Forest Algorithm Results:',
         'Naive Bayes Classifier Results:', 'Support Vector Machine Results:']
accuracy = [predictDTA, predictRFA, predictNBC, predictSVM]
for x, y in zip(title, accuracy):
    title1 = '\033[1m' + x + '\033[0m'
    z = title1.center(73)
    print('')
    print(z)
    print('')
    print('Accuracy:', accuracy_score(data6, y))
    sns.set(font_scale=1.6)
    mat = confusion_matrix(data6, y)
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = True,
               xticklabels = categories,
               yticklabels = categories,
               vmax=150,
               ax = ax)
    plt.title(x[0:23])
    plt.xlabel('True label', size = 20)
    plt.ylabel('Predicted label', size = 20);
    plt.show()
    print('')

In [ ]:
title = ['Decision Tree Algorithm Resulst:', 'Random Forest Algorithm Results:',
         'Naive Bayes Classifier Results:', 'Support Vector Machine Results:']
accuracy = [predictDTA, predictRFA, predictNBC, predictSVM]
for x, y in zip(title, accuracy):
    str = '\033[1m' + x + '\033[0m'
    print('')
    print(str)
    print('')
    print('Accuracy:', "%.2f" % (accuracy_score(data6, y)* 100))